# Ephemeris generation
#### The spacecraft trajectory is extrapolated together with its covariance, ephemerides are returned in a CCSDS OEM format and orbital elements are plotted

In [22]:
""" If you are using this notebook locally, use the following methods to configure your credentials. """
# from fds.config import set_client_id, set_client_secret
# set_client_id('CLIENT_ID')
# set_client_secret('CLIENT_SECRET')

In [23]:
import datetime

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from fds.models.orbit_extrapolation.requests import OemRequest, EphemeridesRequest
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame

In [24]:
object_id = 25544 # NoradID
object_name = "ISS"

tle_iss = TwoLineElement("1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995",
"2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382")

print(f"Latest TLE: {tle_iss.single_line}")
print(f"Latest TLE date (UTC): {tle_iss.date}")
print()

Latest TLE: 1 25544U 98067A   24142.35003124  .00022843  00000-0  38371-3 0  9995
2 25544  51.6390  88.3709 0003333 191.4959 306.2513 15.51667899454382
Latest TLE date (UTC): 2024-05-21 08:24:02.699136+00:00



## Create FDS models

In [25]:
propagation_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.749,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

covariance = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    frame=Frame.TNW
)

# Generate initial OrbitalState
orbital_state = OrbitalState.from_tle(
    tle=tle_iss,
    covariance_matrix=covariance,
    propagation_context=propagation_context,
    spacecraft=spacecraft,
)

# Orbit Data Message request 
oem_request = OemRequest(
    creator="User",
    object_id=str(object_id),
    object_name=object_name,
    frame=Frame.EME2000,
    write_acceleration=False,
    write_covariance=True,
    ephemerides_step=60,
)

ephemerides_request = EphemeridesRequest(
    ephemeris_types=[EphemeridesRequest.EphemerisType.KEPLERIAN],
    step=200,
)



## Perform the use case

In [26]:
target_date = orbital_state.date + datetime.timedelta(
    seconds=orbital_state.mean_orbit.keplerian_period * 3)

oe = OrbitExtrapolation.with_target_date(
    target_date=target_date,
    initial_orbital_state=orbital_state,
    orbit_data_message_request=oem_request,
    ephemerides_request=ephemerides_request,
    extrapolate_covariance=True
)
print("Propagation orbit from TLE date to 3 orbits from now")
print(f"Orbit extrapolation start date: {oe.initial_date}")
print(f"Orbit extrapolation end date: {oe.final_date}")
print(f"Duration: {format(oe.duration / 3600., '.3f')} hours")

Propagation orbit from TLE date to 3 orbits from now
Orbit extrapolation start date: 2024-05-21 08:24:02.699136+00:00
Orbit extrapolation end date: 2024-05-21 13:02:29.173077+00:00
Duration: 4.641 hours


In [27]:
res = oe.run().result

## Print CCSDS Orbit Ephemeris Message
#### Position-velocity ephemeris in inertial frame

In [28]:
print(res.orbit_data_message)

CCSDS_OEM_VERS       = 3.0
CREATION_DATE        = 2024-05-23T09:21:25.734363095
ORIGINATOR           = User


META_START
OBJECT_NAME          = ISS
OBJECT_ID            = 25544
CENTER_NAME          = Earth
REF_FRAME            = EME2000
TIME_SYSTEM          = UTC
START_TIME           = 2024-05-21T08:24:02.699136
STOP_TIME            = 2024-05-21T13:02:29.173077000000195
INTERPOLATION_DEGREE = 4
META_STOP

2024-05-21T08:24:02.699136 -2990.5570593328107 -4925.972550850762 3580.728665596977 3.333953993543136 -5.272859530076239 -4.459736642701176
2024-05-21T08:27:22.699136 -2253.4394650893873 -5846.4367625661835 2605.1314511915193 4.005896731570293 -3.89250134929848 -5.2546477375209895
2024-05-21T08:30:42.699136 -1401.7827549409615 -6469.730273924793 1496.7420940157065 4.474422053476654 -2.3138515813906495 -5.781931269872113
2024-05-21T08:34:02.699136 -478.8606892740573 -6764.124371767692 312.04425699753193 4.715491908880542 -0.6176172052089386 -6.0144499011841965
2024-05-21T08:37:22.69913

### Plotting the results

In [29]:
# %% Extract the ephemerides
ephemerides = res.export_keplerian_ephemeris()

eph_df = pd.DataFrame(ephemerides)

# plot
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Semi-major axis [km]", "Eccentricity [-]", "Inclination [deg]"),
)
# Add traces to the subplots
fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['semi_major_axis'], mode='lines', name='a'), row=1, col=1)
fig.add_trace(go.Scatter(x=eph_df['date'], y=eph_df['eccentricity'], mode='lines', name='e'), row=2, col=1)
fig.add_trace(go.Scatter(x=eph_df['date'], y=np.degrees(eph_df['inclination']), mode='lines', name='i'), row=3, col=1)
# Update layout
fig.update_layout(
    title_text="Osculating Keplerian elements"
)

fig.show()